In [1]:
import requests

r = requests.get('https://www.seshop.com/')
print(type(r))
print(r.status_code)

<class 'requests.models.Response'>
200


In [3]:
text = r.text
for line in text.split('\n'):
    if '<title>' in line or '<h2>' in line:
        print(line.strip())

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>電子書籍ランキング<span class="pull-right"><a href="/ranking/327/"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
<h2>予約ランキング</h2>
<h2>新刊ランキング</h2>
<h2>新刊案内</h2>
<h2>SEshop限定　紙書籍と電子書籍のセット商品</h2>
<h2>商品カテゴリー</h2>
<h2>人気商品ランキング</h2>
<h2>キャンペーン・特集</h2>


In [4]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(text, 'html.parser')
print(soup.title)
print(soup.h2)
print(soup.h2.a["href"])

<title>SEshop｜ 翔泳社の本・電子書籍通販サイト</title>
<h2>書籍ランキング <span class="pull-right"><a href="/ranking/1"><span class="glyphicon glyphicon-chevron-right"></span> もっと見る</a></span></h2>
/ranking/1


In [8]:
atags = soup.find_all('a')
print('aタグの数：', len(atags))
for atag in atags[:5]:
    print('タイトル：', atag.text)
    print('リンク：', atag['href'])

aタグの数： 243
タイトル：  よくある質問
リンク： /help/detail/h337
タイトル：  法人のお客様へ
リンク： /campaign/corp
タイトル：  新規会員登録
リンク： https://www.seshop.com/regist/
タイトル：  ログイン
リンク： #modalLogin
タイトル： 
リンク： /


In [27]:
from datetime import datetime

import requests
from bs4 import BeautifulSoup

r = requests.get("https://www.seshop.com/product/616")
soup = BeautifulSoup(r.text, "html.parser")

books = []  # 各書籍の情報を格納するリスト

# CSSセレクターで <div class="list">の中の<div class="inner"> を取得
divs = soup.select("div.list div.inner")
for div in divs:
    img_url = div.find("img")["src"]
    # 日付の文字列を取得
    day = div.find("span", class_="date").text.strip()
    day = day.replace("発売", "")
    # 日付をdatetimeに変換
    published = datetime.strptime(day, "%Y.%m.%d")
    
    div_txt = div.find("div", class_="txt")
    a_tag = div_txt.find("a")  # aタグを取得
    title = a_tag.text.strip()  # 書籍タイトルを取得
    url = a_tag["href"]  # 書籍URLを取得
    
    # 販売価格を取得
    price_s = div_txt.find("span", class_="sale-price").parent.text
    price_s = price_s.strip()
    price_s = price_s.replace("円（税込）送料無料", "")
    price_s = price_s.replace(",", "")
    price = int(price_s)
    
    book = {
        "title": title,
        "img_url": img_url,
        "url": url,
        "price": price,
        "published": published,
    }
    books.append(book)

In [28]:
books[:3]

[{'title': 'ChatGPTプログラミング2年生 Pythonゲーム作り  体験してわかる！AIと会話でまなべる！',
  'img_url': '/static/images/product/27184/L.png',
  'url': '/product/detail/27184',
  'price': 2640,
  'published': datetime.datetime(2025, 11, 6, 0, 0)},
 {'title': 'Exercise Python  プログラマ脳を鍛える至高の問題集',
  'img_url': '/static/images/product/27150/L.png',
  'url': '/product/detail/27150',
  'price': 2508,
  'published': datetime.datetime(2025, 10, 16, 0, 0)},
 {'title': 'Pythonレベルアップドリル  初心者から一歩進むための厳選問題集',
  'img_url': '/static/images/product/27014/L.png',
  'url': '/product/detail/27014',
  'price': 2530,
  'published': datetime.datetime(2025, 8, 5, 0, 0)}]

In [29]:
import pandas as pd

df = pd.DataFrame(books)
df.head()

,title,img_url,url,price,published
0,ChatGPTプログラミング2年生 Pythonゲーム作り 体験してわかる！AIと会話でま...,/static/images/product/27184/L.png,/product/detail/27184,2640,2025-11-06
1,Exercise Python プログラマ脳を鍛える至高の問題集,/static/images/product/27150/L.png,/product/detail/27150,2508,2025-10-16
2,Pythonレベルアップドリル 初心者から一歩進むための厳選問題集,/static/images/product/27014/L.png,/product/detail/27014,2530,2025-08-05
3,Pythonによるあたらしいデータ分析の教科書 第3版,/static/images/product/26879/L.png,/product/detail/26879,3058,2025-05-21
4,独習Python 第2版,/static/images/product/26866/L.png,/product/detail/26866,3608,2025-05-14
